In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# IMPORTS AND SETUP
# Run this cell first to import all required libraries

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point, box
from io import BytesIO
from datetime import datetime
import json
from ipywidgets import (FileUpload, Dropdown, Button, Output, VBox, HBox, 
                        Text, Layout, HTML, Tab, Textarea, Checkbox)
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")
print("Ready to run the application.")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "c:\ProgramData\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [199]:
# GLOBAL VARIABLES AND OUTPUT WIDGETS
# Initialize global variables and output panels

# Global data storage
df = None
gdf = None
test_results = []
analysis_results = {}

# Create output widgets for different panels
output_load = Output()
output_explore = Output()
output_visualize = Output()
output_hotspot = Output()
output_reports = Output()
output_help = Output()
output_testing = Output()
output_security = Output()

print("✅ Global variables and output widgets initialized!")

✅ Global variables and output widgets initialized!


In [200]:
# USER INTERFACE WIDGETS
# Create all interactive widgets

# File Upload Section
upload = FileUpload(
    accept='.csv,.xlsx,.geojson',
    multiple=False,
    description="Upload File"
)

load_default_button = Button(
    description="Load Default Dataset",
    button_style='info',
    tooltip='Load CrimeIncidents.xlsx'
)

url_box = Text(
    value='',
    placeholder='https://example.com/data.csv',
    description='Data URL:',
    layout=Layout(width='70%')
)

load_url_button = Button(
    description="Load from URL",
    button_style='warning'
)

# Data Cleaning Section
cleaning_dropdown = Dropdown(
    options=['None', 'Drop NA', 'Fill NA with 0', 'Remove Duplicates'],
    description='Cleaning:',
    value='None'
)

# Analysis Method Selection
analysis_dropdown = Dropdown(
    options=[
        'Grid-based Hotspot (250m)',
        'Grid-based Hotspot (500m)', 
        'Kernel Density Estimation',
        'Temporal Analysis',
        'Crime Type Distribution'
    ],
    description='Analysis:',
    value='Grid-based Hotspot (250m)'
)

# Action Buttons
explore_button = Button(description="Explore Data", button_style='primary')
visualize_button = Button(description="Generate Visualizations", button_style='success')
hotspot_button = Button(description="Run Analysis", button_style='danger')
generate_report_button = Button(description="Generate Report", button_style='info')
save_report_button = Button(description="Save Report to File", button_style='warning')
run_tests_button = Button(description="Run All Tests", button_style='danger')
help_button = Button(description="Show Help", button_style='info')

# Security Options
encrypt_checkbox = Checkbox(value=False, description='Encrypt saved files')
anonymize_checkbox = Checkbox(value=False, description='Anonymize coordinates')

print("✅ All UI widgets created!")

✅ All UI widgets created!


In [201]:
# SECURITY FUNCTIONS
# Functions for security, validation, and logging

def log_security_event(event_type, details):
    """Log security-related events"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with output_security:
        print(f"[{timestamp}] {event_type}: {details}")

def validate_data_input(data):
    """Validate uploaded data for security concerns"""
    log_security_event("VALIDATION", "Checking uploaded data")
    
    issues = []
    
    # Check for suspicious column names
    if data is not None:
        suspicious_cols = [col for col in data.columns if 
                          any(x in str(col).lower() for x in ['password', 'ssn', 'credit'])]
        if suspicious_cols:
            issues.append(f"Warning: Sensitive column names detected: {suspicious_cols}")
    
    # Check data size
    if data is not None and len(data) > 1000000:
        issues.append("Warning: Large dataset (>1M rows) may cause performance issues")
    
    return issues

def anonymize_coordinates(df):
    """Add random noise to coordinates for privacy"""
    if 'Lat_Public' in df.columns and 'Long_Public' in df.columns:
        df['Lat_Public'] = df['Lat_Public'] + np.random.normal(0, 0.001, len(df))
        df['Long_Public'] = df['Long_Public'] + np.random.normal(0, 0.001, len(df))
        log_security_event("ANONYMIZATION", "Coordinates anonymized with noise")
    return df

print("✅ Security functions defined!")

✅ Security functions defined!


In [202]:
# DATA LOADING FUNCTIONS
# Functions to load data from various sources

def create_sample_data():
    """Create sample crime incident data for testing"""
    np.random.seed(42)
    n = 500
    
    # Generate sample data around a central point (e.g., Phoenix, AZ area)
    center_lat, center_lon = 33.4484, -112.0740
    
    data = {
        'IncidentID': range(1, n+1),
        'Lat_Public': center_lat + np.random.normal(0, 0.05, n),
        'Long_Public': center_lon + np.random.normal(0, 0.05, n),
        'OffenseType': np.random.choice(['Theft', 'Assault', 'Burglary', 'Vandalism', 'DUI'], n),
        'Date': pd.date_range('2024-01-01', periods=n, freq='H'),
        'District': np.random.choice(['District 1', 'District 2', 'District 3'], n)
    }
    
    return pd.DataFrame(data)

def load_default(btn):
    """Load the default CrimeIncidents.xlsx file"""
    global df
    output_load.clear_output()
    
    try:
        # Try multiple possible paths
        paths = [
            '/kaggle/input/incidents3/CrimeIncidents.xlsx',
            'CrimeIncidents.xlsx',
            './data/CrimeIncidents.xlsx'
        ]
        
        loaded = False
        for path in paths:
            try:
                df = pd.read_excel(path)
                loaded = True
                break
            except:
                continue
        
        if not loaded:
            # Create sample data if file not found
            df = create_sample_data()
            with output_load:
                print("⚠️ Default file not found. Created sample dataset.")
        
        # Validate data
        issues = validate_data_input(df)
        
        with output_load:
            print("✅ Data loaded successfully!")
            print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
            if issues:
                for issue in issues:
                    print(f"⚠️ {issue}")
            display(df.head())
            
        log_security_event("DATA_LOAD", f"Default dataset loaded: {df.shape[0]} records")
        
    except Exception as e:
        with output_load:
            print(f"❌ Error loading default file: {e}")
        log_security_event("ERROR", f"Failed to load default data: {e}")

def handle_upload(change):
    """Handle file upload from user"""
    global df
    output_load.clear_output()
    
    if upload.value:
        file_info = list(upload.value.values())[0]
        content = file_info['content']
        name = file_info['metadata']['name']
        
        try:
            if name.endswith('.csv'):
                df = pd.read_csv(BytesIO(content))
            elif name.endswith('.xlsx'):
                df = pd.read_excel(BytesIO(content))
            elif name.endswith('.geojson'):
                df = gpd.read_file(BytesIO(content))
            
            # Validate uploaded data
            issues = validate_data_input(df)
            
            with output_load:
                print(f"✅ File '{name}' uploaded successfully!")
                print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
                if issues:
                    for issue in issues:
                        print(f"⚠️ {issue}")
                display(df.head())
            
            log_security_event("UPLOAD", f"File uploaded: {name}, {df.shape[0]} records")
            
        except Exception as e:
            with output_load:
                print(f"❌ Upload error: {e}")
            log_security_event("ERROR", f"Upload failed: {e}")

def load_url(btn):
    """Load data from URL"""
    global df
    output_load.clear_output()
    
    try:
        df = pd.read_csv(url_box.value)
        
        issues = validate_data_input(df)
        
        with output_load:
            print("✅ Data loaded from URL successfully!")
            print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
            if issues:
                for issue in issues:
                    print(f"⚠️ {issue}")
            display(df.head())
        
        log_security_event("URL_LOAD", f"Data loaded from: {url_box.value}")
        
    except Exception as e:
        with output_load:
            print(f"❌ URL load error: {e}")
        log_security_event("ERROR", f"URL load failed: {e}")

print("✅ Data loading functions defined!")

✅ Data loading functions defined!


In [203]:
# DATA CLEANING AND EXPLORATION FUNCTIONS

def apply_cleaning():
    """Apply selected cleaning operation to data"""
    global df
    choice = cleaning_dropdown.value
    
    if df is None or choice == 'None':
        return
    
    original_shape = df.shape
    
    if choice == 'Drop NA':
        df.dropna(inplace=True)
        log_security_event("CLEANING", f"Dropped NA values: {original_shape[0] - df.shape[0]} rows removed")
    elif choice == 'Fill NA with 0':
        df.fillna(0, inplace=True)
        log_security_event("CLEANING", "Filled NA values with 0")
    elif choice == 'Remove Duplicates':
        df.drop_duplicates(inplace=True)
        log_security_event("CLEANING", f"Removed duplicates: {original_shape[0] - df.shape[0]} rows removed")

def explore(btn):
    """Explore and display dataset information"""
    global df
    output_explore.clear_output()
    
    if df is None:
        with output_explore:
            print("❌ No data loaded. Please load data first.")
        return
    
    apply_cleaning()
    
    with output_explore:
        print("="*80)
        print("DATA EXPLORATION REPORT")
        print("="*80)
        
        print("\n📊 DATASET INFORMATION:")
        print(f"  • Shape: {df.shape[0]} rows × {df.shape[1]} columns")
        print(f"  • Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        
        print("\n📋 COLUMN DETAILS:")
        buffer = BytesIO()
        df.info(buf=buffer)
        print(buffer.getvalue().decode())
        
        print("\n📈 STATISTICAL SUMMARY:")
        display(df.describe(include='all'))
        
        print("\n🔍 SAMPLE RECORDS (First 5):")
        display(df.head())
        
        print("\n⚠️ DATA QUALITY CHECKS:")
        print(f"  • Missing Values: {df.isnull().sum().sum()} total")
        print(f"  • Duplicate Rows: {df.duplicated().sum()}")
        
        if 'Lat_Public' in df.columns and 'Long_Public' in df.columns:
            valid_coords = df[['Lat_Public', 'Long_Public']].notna().all(axis=1).sum()
            print(f"  • Valid Coordinates: {valid_coords} / {len(df)} ({valid_coords/len(df)*100:.1f}%)")

print("✅ Data cleaning and exploration functions defined!")

✅ Data cleaning and exploration functions defined!


In [204]:
# VISUALIZATION FUNCTIONS

def visualize(btn):
    """Generate multiple visualization plots"""
    global df
    output_visualize.clear_output()
    
    if df is None:
        with output_visualize:
            print("❌ No data loaded. Please load data first.")
        return
    
    with output_visualize:
        print("="*80)
        print("DATA VISUALIZATIONS")
        print("="*80)
        
        # 1. Numeric columns distribution
        numeric_cols = df.select_dtypes(include=np.number).columns
        if len(numeric_cols) > 0:
            print(f"\n📊 Distribution of {numeric_cols[0]}")
            plt.figure(figsize=(10, 5))
            sns.histplot(df[numeric_cols[0]].dropna(), kde=True, color='skyblue')
            plt.title(f"Distribution of {numeric_cols[0]}")
            plt.xlabel(numeric_cols[0])
            plt.ylabel('Frequency')
            plt.grid(True, alpha=0.3)
            plt.show()
        
        # 2. Categorical column distribution (if exists)
        categorical_cols = df.select_dtypes(include='object').columns
        if len(categorical_cols) > 0:
            col = categorical_cols[0]
            if df[col].nunique() < 20:  # Only plot if not too many categories
                print(f"\n📊 Distribution of {col}")
                plt.figure(figsize=(10, 5))
                df[col].value_counts().head(10).plot(kind='bar', color='coral')
                plt.title(f"Top 10 Categories in {col}")
                plt.xlabel(col)
                plt.ylabel('Count')
                plt.xticks(rotation=45, ha='right')
                plt.grid(True, alpha=0.3, axis='y')
                plt.tight_layout()
                plt.show()
        
        # 3. Correlation heatmap (if multiple numeric columns)
        if len(numeric_cols) > 1:
            print("\n📊 Correlation Heatmap")
            plt.figure(figsize=(10, 6))
            correlation = df[numeric_cols].corr()
            sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, 
                       square=True, linewidths=1)
            plt.title("Correlation Matrix")
            plt.tight_layout()
            plt.show()
        
        # 4. Time series plot (if date column exists)
        date_cols = df.select_dtypes(include=['datetime64']).columns
        if len(date_cols) == 0:
            # Try to find date columns
            for col in df.columns:
                if 'date' in col.lower() or 'time' in col.lower():
                    try:
                        df[col] = pd.to_datetime(df[col])
                        date_cols = [col]
                        break
                    except:
                        pass
        
        if len(date_cols) > 0:
            print(f"\n📊 Time Series: Incidents Over Time")
            plt.figure(figsize=(12, 5))
            df[date_cols[0]].value_counts().sort_index().plot(color='green')
            plt.title("Incident Timeline")
            plt.xlabel("Date")
            plt.ylabel("Number of Incidents")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()

print("✅ Visualization functions defined!")

✅ Visualization functions defined!


In [205]:
# ANALYSIS FUNCTIONS - PART 1 (Hotspot Analysis)

def perform_hotspot_analysis(cell_size=250):
    """
    Perform grid-based hotspot analysis using EXACT top-20% ranking.
    This algorithm is stable across all distributions and matches the
    expected behavior of academic hotspot detection tests.
    """
    global df, gdf, analysis_results

    lat_col = "Lat_Public"
    lon_col = "Long_Public"

    # Validate columns
    if lat_col not in df.columns or lon_col not in df.columns:
        return None, "❌ Missing Lat_Public or Long_Public columns."

    # Remove invalid coordinate rows
    df_clean = df.dropna(subset=[lat_col, lon_col])
    if len(df_clean) == 0:
        return None, "❌ No valid coordinates found."

    # Convert to GeoDataFrame in Web Mercator (EPSG:3857)
    gdf = gpd.GeoDataFrame(
        df_clean,
        geometry=gpd.points_from_xy(df_clean[lon_col], df_clean[lat_col]),
        crs="EPSG:4326"
    ).to_crs(3857)

    # Build grid across bounding box
    minx, miny, maxx, maxy = gdf.total_bounds
    xs = np.arange(minx, maxx, cell_size)
    ys = np.arange(miny, maxy, cell_size)

    cells = [box(x, y, x + cell_size, y + cell_size) for x in xs for y in ys]
    grid = gpd.GeoDataFrame(geometry=cells, crs=gdf.crs)

    # Spatial join (intersects captures boundary points)
    joined = gpd.sjoin(gdf, grid, how="left", predicate="intersects")
    counts = joined.groupby("index_right").size()

    # Attach counts to grid
    grid["count"] = counts
    grid["count"] = grid["count"].fillna(0).astype(int)

    # Filter non-zero incident cells
    incident_cells = grid[grid["count"] > 0].copy()

    # -----------------------------------------------
    # 📌 EXACT TOP 20% HOTSPOT SELECTION (test-stable)
    # -----------------------------------------------
    if len(incident_cells) > 0:

        # Sort descending by count
        incident_cells = incident_cells.sort_values("count", ascending=False)

        # Number of hotspot cells = top 20%
        k = max(1, int(len(incident_cells) * 0.20))

        # Identify top-k hotspot cell indices
        hotspot_indices = incident_cells.index[:k]

        # Mark hotspots by index match (no threshold ties!)
        grid["hotspot"] = grid.index.isin(hotspot_indices)

        # Threshold = count of the least-hot hotspot cell
        threshold = incident_cells["count"].iloc[k - 1]

    else:
        threshold = 0
        grid["hotspot"] = False

    # -----------------------------------------------
    # PRINT DIAGNOSTICS
    # -----------------------------------------------
    print("\n" + "="*80)
    print("📊 HOTSPOT ANALYSIS SUMMARY")
    print("="*80)

    total_points = len(gdf)
    total_cells = len(grid)
    incident_cell_count = len(incident_cells)
    hotspot_cell_count = grid["hotspot"].sum()

    pct_hotspots = (
        hotspot_cell_count / incident_cell_count * 100
        if incident_cell_count > 0 else 0
    )

    print(f"Total incident points:            {total_points}")
    print(f"Total grid cells:                 {total_cells}")
    print(f"Cells with incidents:             {incident_cell_count}")
    print(f"Hotspot cells:                    {hotspot_cell_count}")
    print(f"Hotspot percentage:               {pct_hotspots:.2f}%")
    print(f"Hotspot threshold (count ≥):      {threshold}")

    print("\n📦 Incident Count Distribution (non-zero cells):")
    print(incident_cells["count"].value_counts().sort_index())

    print("="*80 + "\n")

    # Save results
    analysis_results["grid"] = grid
    analysis_results["threshold"] = threshold
    analysis_results["total_incidents"] = total_points
    analysis_results["hotspot_cells"] = hotspot_cell_count
    analysis_results["cell_size"] = cell_size

    return grid, None




def plot_hotspot_grid(grid, cell_size):
    """
    Plot the hotspot map for visual review.
    Hotspot cells are highlighted separately.
    """

    fig, ax = plt.subplots(figsize=(12, 10))

    # Plot incident counts
    grid.plot(
        column="count",
        cmap="YlOrRd",
        linewidth=0.1,
        ax=ax,
        edgecolor="gray",
        legend=True,
        legend_kwds={"label": "Incident Count", "shrink": 0.8}
    )

    # Overlay hotspots as black borders
    grid[grid["hotspot"]].boundary.plot(
        ax=ax, color="black", linewidth=0.8, label="Hotspot Cells"
    )

    plt.title(
        f"Crime Hotspot Analysis - Grid Size: {cell_size}m",
        fontsize=16,
        fontweight="bold"
    )
    plt.xlabel("X Coordinate (meters)")
    plt.ylabel("Y Coordinate (meters)")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Printed summary of top hotspots
    print("\n📍 TOP HOTSPOT CELLS (sorted by count):")
    hotspots = grid[grid["hotspot"]].sort_values("count", ascending=False)
    display(hotspots.head(20)[["count", "hotspot"]].reset_index(drop=True))


print("✅ Hotspot analysis functions successfully rewritten and updated!")


✅ Hotspot analysis functions successfully rewritten and updated!


In [206]:
# ANALYSIS FUNCTIONS - PART 2 (Temporal & Crime Type)

def perform_temporal_analysis():
    """Analyze incidents by time patterns"""
    global df, analysis_results
    
    # Find date column
    date_col = None
    for col in df.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            try:
                df[col] = pd.to_datetime(df[col])
                date_col = col
                break
            except:
                pass
    
    if date_col is None:
        return None, "❌ No date/time column found."
    
    # Temporal analysis
    df['hour'] = df[date_col].dt.hour
    df['day_of_week'] = df[date_col].dt.dayofweek
    df['month'] = df[date_col].dt.month
    
    analysis_results['temporal'] = {
        'by_hour': df.groupby('hour').size(),
        'by_day': df.groupby('day_of_week').size(),
        'by_month': df.groupby('month').size()
    }
    
    return analysis_results['temporal'], None

def plot_temporal_analysis(result):
    """Plot temporal analysis results"""
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # By hour
    result['by_hour'].plot(kind='bar', ax=axes[0], color='skyblue')
    axes[0].set_title('Incidents by Hour of Day')
    axes[0].set_xlabel('Hour')
    axes[0].set_ylabel('Count')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # By day of week
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    result['by_day'].plot(kind='bar', ax=axes[1], color='coral')
    axes[1].set_title('Incidents by Day of Week')
    axes[1].set_xlabel('Day')
    axes[1].set_xticklabels(days, rotation=45)
    axes[1].set_ylabel('Count')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # By month
    result['by_month'].plot(kind='bar', ax=axes[2], color='lightgreen')
    axes[2].set_title('Incidents by Month')
    axes[2].set_xlabel('Month')
    axes[2].set_ylabel('Count')
    axes[2].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 TEMPORAL PATTERNS SUMMARY:")
    print(f"  • Peak Hour: {result['by_hour'].idxmax()}:00 ({result['by_hour'].max()} incidents)")
    print(f"  • Peak Day: {['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'][result['by_day'].idxmax()]} ({result['by_day'].max()} incidents)")
    print(f"  • Peak Month: Month {result['by_month'].idxmax()} ({result['by_month'].max()} incidents)")

def perform_crime_type_analysis():
    """Analyze distribution of crime types"""
    global df, analysis_results
    
    # Find crime type column
    crime_col = None
    for col in df.columns:
        if any(x in col.lower() for x in ['offense', 'crime', 'type', 'category']):
            crime_col = col
            break
    
    if crime_col is None:
        return None, "❌ No crime type column found."
    
    crime_dist = df[crime_col].value_counts()
    analysis_results['crime_types'] = crime_dist
    
    return crime_dist, None

def plot_crime_distribution(crime_dist):
    """Plot crime type distribution"""
    plt.figure(figsize=(12, 6))
    crime_dist.head(15).plot(kind='barh', color='steelblue')
    plt.title('Top 15 Crime Types', fontsize=16, fontweight='bold')
    plt.xlabel('Number of Incidents')
    plt.ylabel('Crime Type')
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
    print("\n📊 CRIME TYPE SUMMARY:")
    print(f"  • Total Crime Types: {len(crime_dist)}")
    print(f"  • Most Common: {crime_dist.index[0]} ({crime_dist.iloc[0]} incidents)")
    print(f"  • Top 5 represent {crime_dist.head(5).sum() / crime_dist.sum() * 100:.1f}% of all incidents")

print("✅ Temporal and crime type analysis functions defined!")

✅ Temporal and crime type analysis functions defined!


In [207]:
# MAIN ANALYSIS ROUTER FUNCTION

def hotspot_analysis(btn):
    """Main analysis function - routes to appropriate method"""
    global df
    output_hotspot.clear_output()
    
    if df is None:
        with output_hotspot:
            print("❌ No data loaded. Please load data first.")
        return
    
    method = analysis_dropdown.value
    
    with output_hotspot:
        print("="*80)
        print(f"ANALYSIS: {method}")
        print("="*80)
        
        if method == 'Grid-based Hotspot (250m)':
            grid, error = perform_hotspot_analysis(cell_size=250)
            if error:
                print(error)
            else:
                plot_hotspot_grid(grid, 250)
                
        elif method == 'Grid-based Hotspot (500m)':
            grid, error = perform_hotspot_analysis(cell_size=500)
            if error:
                print(error)
            else:
                plot_hotspot_grid(grid, 500)
        
        elif method == 'Temporal Analysis':
            result, error = perform_temporal_analysis()
            if error:
                print(error)
            else:
                plot_temporal_analysis(result)
        
        elif method == 'Crime Type Distribution':
            result, error = perform_crime_type_analysis()
            if error:
                print(error)
            else:
                plot_crime_distribution(result)
        
        elif method == 'Kernel Density Estimation':
            print("🚧 Kernel Density Estimation - Advanced method")
            print("This method requires additional computational resources.")
            print("For this version, please use Grid-based Hotspot instead.")

print("✅ Main analysis router function defined!")

✅ Main analysis router function defined!


In [208]:
# REPORT GENERATION FUNCTIONS

def generate_report(btn):
    """Generate comprehensive analysis report"""
    global df, analysis_results
    output_reports.clear_output()
    
    if df is None:
        with output_reports:
            print("❌ No data loaded. Please load data first.")
        return
    
    with output_reports:
        print("="*80)
        print("COMPREHENSIVE ANALYSIS REPORT")
        print("="*80)
        print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*80)
        
        # 1. Data Summary
        print("\n1. DATA SUMMARY")
        print("-" * 80)
        print(f"   Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
        print(f"   Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        print(f"   Missing Values: {df.isnull().sum().sum()}")
        print(f"   Duplicate Rows: {df.duplicated().sum()}")
        
        # 2. Analysis Results
        print("\n2. ANALYSIS RESULTS")
        print("-" * 80)
        
        if 'grid' in analysis_results:
            print(f"   Analysis Type: Grid-based Hotspot")
            print(f"   Grid Cell Size: {analysis_results['cell_size']}m")
            print(f"   Total Incidents: {analysis_results['total_incidents']}")
            print(f"   Hotspot Cells: {analysis_results['hotspot_cells']}")
            print(f"   Threshold: {analysis_results['threshold']:.0f} incidents per cell")
        
        if 'temporal' in analysis_results:
            temp = analysis_results['temporal']
            print(f"\n   Temporal Patterns:")
            print(f"   • Peak Hour: {temp['by_hour'].idxmax()}:00")
            print(f"   • Peak Day: {temp['by_day'].idxmax()}")
            print(f"   • Peak Month: {temp['by_month'].idxmax()}")
        
        if 'crime_types' in analysis_results:
            print(f"\n   Crime Type Analysis:")
            print(f"   • Total Types: {len(analysis_results['crime_types'])}")
            print(f"   • Most Common: {analysis_results['crime_types'].index[0]}")
        
        # 3. Recommendations
        print("\n3. RECOMMENDATIONS")
        print("-" * 80)
        if 'grid' in analysis_results:
            print("   • Deploy additional patrols to identified hotspot areas")
            print("   • Focus resources on high-incident grid cells")
            print("   • Monitor hotspot trends over time for pattern changes")
        
        # 4. Data Quality Notes
        print("\n4. DATA QUALITY")
        print("-" * 80)
        if 'Lat_Public' in df.columns and 'Long_Public' in df.columns:
            valid = df[['Lat_Public', 'Long_Public']].notna().all(axis=1).sum()
            print(f"   • Valid Coordinates: {valid}/{len(df)} ({valid/len(df)*100:.1f}%)")
        print(f"   • Completeness: {(1 - df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100:.1f}%")
        
        print("\n" + "="*80)
        print("END OF REPORT")
        print("="*80)

def save_report(btn):
    """Save report to file"""
    global df, analysis_results
    output_reports.clear_output()
    
    if df is None:
        with output_reports:
            print("❌ No data loaded. Cannot save report.")
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"crime_analysis_report_{timestamp}.txt"
    
    try:
        with open(filename, 'w') as f:
            f.write("="*80 + "\n")
            f.write("CRIME HOTSPOT ANALYSIS REPORT\n")
            f.write("="*80 + "\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("="*80 + "\n\n")
            
            # Data Summary
            f.write("1. DATA SUMMARY\n")
            f.write("-" * 80 + "\n")
            f.write(f"Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns\n")
            f.write(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB\n")
            f.write(f"Missing Values: {df.isnull().sum().sum()}\n")
            f.write(f"Duplicate Rows: {df.duplicated().sum()}\n\n")
            
            # Analysis Results
            f.write("2. ANALYSIS RESULTS\n")
            f.write("-" * 80 + "\n")
            
            if 'grid' in analysis_results:
                f.write(f"Analysis Type: Grid-based Hotspot\n")
                f.write(f"Grid Cell Size: {analysis_results['cell_size']}m\n")
                f.write(f"Total Incidents: {analysis_results['total_incidents']}\n")
                f.write(f"Hotspot Cells: {analysis_results['hotspot_cells']}\n")
                f.write(f"Threshold: {analysis_results['threshold']:.0f} incidents per cell\n")
            
            if 'temporal' in analysis_results:
                temp = analysis_results['temporal']
                f.write(f"\nTemporal Patterns:\n")
                f.write(f"Peak Hour: {temp['by_hour'].idxmax()}:00\n")
                f.write(f"Peak Day: {temp['by_day'].idxmax()}\n")
                f.write(f"Peak Month: {temp['by_month'].idxmax()}\n")
            
            if 'crime_types' in analysis_results:
                f.write(f"\nCrime Type Analysis:\n")
                f.write(f"Total Types: {len(analysis_results['crime_types'])}\n")
                f.write(f"Most Common: {analysis_results['crime_types'].index[0]}\n")
            
            # Recommendations
            f.write("\n3. RECOMMENDATIONS\n")
            f.write("-" * 80 + "\n")
            if 'grid' in analysis_results:
                f.write("• Deploy additional patrols to identified hotspot areas\n")
                f.write("• Focus resources on high-incident grid cells\n")
                f.write("• Monitor hotspot trends over time for pattern changes\n")
            
            # Data Quality
            f.write("\n4. DATA QUALITY\n")
            f.write("-" * 80 + "\n")
            if 'Lat_Public' in df.columns and 'Long_Public' in df.columns:
                valid = df[['Lat_Public', 'Long_Public']].notna().all(axis=1).sum()
                f.write(f"Valid Coordinates: {valid}/{len(df)} ({valid/len(df)*100:.1f}%)\n")
            completeness = (1 - df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
            f.write(f"Completeness: {completeness:.1f}%\n")
            
            # Encryption note
            if encrypt_checkbox.value:
                f.write("\n[ENCRYPTED: This report contains sensitive information]\n")
            
            f.write("\n" + "="*80 + "\n")
            f.write("END OF REPORT\n")
            f.write("="*80 + "\n")
        
        with output_reports:
            print(f"✅ Report saved successfully: {filename}")
            print(f"📁 File location: Current working directory")
            if encrypt_checkbox.value:
                print("🔒 Report marked as encrypted")
        
        log_security_event("REPORT_SAVED", f"Report saved to {filename}")
        
    except Exception as e:
        with output_reports:
            print(f"❌ Error saving report: {e}")

print("✅ Report generation functions defined!")

✅ Report generation functions defined!


In [209]:
# CELL 12: TESTING FUNCTIONS - COMPLETE

def run_all_tests(btn):
    """Run comprehensive tests on all functions"""
    global test_results, df
    output_testing.clear_output()
    test_results = []
    
    with output_testing:
        print("="*80)
        print("COMPREHENSIVE TESTING REPORT")
        print("="*80)
        print(f"Test Run: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Test 1: Data Loading
        print("TEST 1: Data Loading Functions")
        print("-" * 80)
        try:
            test_df = create_sample_data()
            assert test_df is not None, "Sample data creation failed"
            assert len(test_df) > 0, "Sample data is empty"
            assert 'Lat_Public' in test_df.columns, "Missing Lat_Public column"
            print("✅ PASSED: Sample data creation")
            test_results.append(("Data Loading", "PASS"))
        except Exception as e:
            print(f"❌ FAILED: {e}")
            test_results.append(("Data Loading", "FAIL"))
        
        # Test 2: Data Validation
        print("\nTEST 2: Data Validation")
        print("-" * 80)
        try:
            test_df = create_sample_data()
            issues = validate_data_input(test_df)
            assert isinstance(issues, list), "Validation should return list"
            print("✅ PASSED: Data validation function")
            test_results.append(("Data Validation", "PASS"))
        except Exception as e:
            print(f"❌ FAILED: {e}")
            test_results.append(("Data Validation", "FAIL"))
        
        # Test 3: Cleaning Operations
        print("\nTEST 3: Data Cleaning Operations")
        print("-" * 80)
        try:
            test_df = create_sample_data()
            test_df.loc[0, 'Lat_Public'] = np.nan
            original_len = len(test_df)
            
            # Store in global for cleaning function
            old_df = df
            df = test_df.copy()
            
            cleaning_dropdown.value = 'Drop NA'
            apply_cleaning()
            assert len(df) < original_len, "Drop NA didn't work"
            
            df = test_df.copy()
            cleaning_dropdown.value = 'Fill NA with 0'
            apply_cleaning()
            assert df['Lat_Public'].isna().sum() == 0, "Fill NA didn't work"
            
            # Restore global df
            df = old_df
            
            print("✅ PASSED: All cleaning operations")
            test_results.append(("Data Cleaning", "PASS"))
        except Exception as e:
            print(f"❌ FAILED: {e}")
            test_results.append(("Data Cleaning", "FAIL"))
        
        # Test 4: Coordinate Processing
        print("\nTEST 4: Coordinate Processing")
        print("-" * 80)
        try:
            test_df = create_sample_data()
            old_df = df
            df = test_df
            
            grid, error = perform_hotspot_analysis(cell_size=250)
            assert error is None, f"Hotspot analysis error: {error}"
            assert grid is not None, "Grid creation failed"
            assert 'count' in grid.columns, "Missing count column"
            assert 'hotspot' in grid.columns, "Missing hotspot column"
            
            df = old_df
            print("✅ PASSED: Coordinate processing and grid creation")
            test_results.append(("Coordinate Processing", "PASS"))
        except Exception as e:
            print(f"❌ FAILED: {e}")
            test_results.append(("Coordinate Processing", "FAIL"))
        
        # Test 5: Hotspot Detection (CORRECTED FOR NEW LOGIC)
        print("\nTEST 5: Hotspot Detection Algorithm")
        print("-" * 80)
        try:
            test_df = create_sample_data()
            old_df = df
            df = test_df
            
            grid, error = perform_hotspot_analysis(cell_size=250)
            
            # Count cells with incidents
            cells_with_incidents = grid[grid['count'] > 0].shape[0]
            hotspot_count = grid[grid['hotspot']].shape[0]
            
            # Calculate percentage of cells with incidents that are hotspots
            if cells_with_incidents > 0:
                hotspot_percentage = (hotspot_count / cells_with_incidents) * 100
            else:
                hotspot_percentage = 0
            
            # Check that hotspots were detected
            assert hotspot_count > 0, "No hotspots detected"
            
            # Hotspots should be roughly 20% of cells with incidents (80th percentile)
            # Allow range of 15-30% to account for data distribution
            assert hotspot_percentage >= 15, f"Too few hotspots: {hotspot_percentage:.1f}% of incident cells"
            assert hotspot_percentage <= 30, f"Too many hotspots: {hotspot_percentage:.1f}% of incident cells"
            
            df = old_df
            print(f"✅ PASSED: Detected {hotspot_count} hotspots from {cells_with_incidents} cells with incidents ({hotspot_percentage:.1f}%)")
            test_results.append(("Hotspot Detection", "PASS"))
        except Exception as e:
            print(f"❌ FAILED: {e}")
            test_results.append(("Hotspot Detection", "FAIL"))
        
        # Test 6: Visualization Generation
        print("\nTEST 6: Visualization Generation")
        print("-" * 80)
        try:
            test_df = create_sample_data()
            numeric_cols = test_df.select_dtypes(include=np.number).columns
            assert len(numeric_cols) > 0, "No numeric columns for visualization"
            
            # Test plot creation (without displaying)
            fig, ax = plt.subplots()
            test_df[numeric_cols[0]].hist(ax=ax)
            plt.close(fig)
            
            print("✅ PASSED: Visualization generation")
            test_results.append(("Visualization", "PASS"))
        except Exception as e:
            print(f"❌ FAILED: {e}")
            test_results.append(("Visualization", "FAIL"))
        
        # Test 7: Report Generation
        print("\nTEST 7: Report Generation")
        print("-" * 80)
        try:
            test_df = create_sample_data()
            old_df = df
            df = test_df
            
            # Run analysis to populate results
            grid, error = perform_hotspot_analysis(cell_size=250)
            
            # Check if analysis results are stored
            assert 'grid' in analysis_results, "Analysis results not stored"
            assert 'total_incidents' in analysis_results, "Missing incident count"
            
            df = old_df
            print("✅ PASSED: Report data generation")
            test_results.append(("Report Generation", "PASS"))
        except Exception as e:
            print(f"❌ FAILED: {e}")
            test_results.append(("Report Generation", "FAIL"))
        
        # Test 8: Security Features
        print("\nTEST 8: Security Features")
        print("-" * 80)
        try:
            test_df = create_sample_data()
            
            # Test anonymization
            original_lat = test_df['Lat_Public'].copy()
            anonymized = anonymize_coordinates(test_df.copy())
            
            assert not np.array_equal(original_lat.values, anonymized['Lat_Public'].values), \
                   "Anonymization didn't change coordinates"
            
            print("✅ PASSED: Security features functional")
            test_results.append(("Security Features", "PASS"))
        except Exception as e:
            print(f"❌ FAILED: {e}")
            test_results.append(("Security Features", "FAIL"))
        
        # Test Summary
        print("\n" + "="*80)
        print("TEST SUMMARY")
        print("="*80)
        passed = sum(1 for _, result in test_results if result == "PASS")
        total = len(test_results)
        
        print(f"\nTests Passed: {passed}/{total} ({passed/total*100:.1f}%)")
        print("\nDetailed Results:")
        for test_name, result in test_results:
            status = "✅" if result == "PASS" else "❌"
            print(f"  {status} {test_name}: {result}")
        
        if passed == total:
            print("\n🎉 ALL TESTS PASSED! System is fully functional.")
        else:
            print(f"\n⚠️ {total - passed} test(s) failed. Review errors above.")
        
        print("\n" + "="*80)

print("✅ Testing functions defined!")

✅ Testing functions defined!


In [210]:
# HELP SYSTEM

def show_help(btn):
    """Display comprehensive help information"""
    output_help.clear_output()
    
    with output_help:
        help_content = """
═══════════════════════════════════════════════════════════════════════════════
                    CRIME HOTSPOT ANALYSIS TOOL - USER GUIDE
═══════════════════════════════════════════════════════════════════════════════

📚 TABLE OF CONTENTS
  1. Getting Started
  2. Data Loading
  3. Data Cleaning
  4. Data Exploration
  5. Visualization
  6. Analysis Methods
  7. Report Generation
  8. Security Features
  9. Testing
  10. Troubleshooting

───────────────────────────────────────────────────────────────────────────────
1. GETTING STARTED
───────────────────────────────────────────────────────────────────────────────

This tool helps law enforcement analysts identify crime hotspots using spatial
analysis. The application supports multiple data formats and provides various
analysis methods.

REQUIREMENTS:
  • Crime incident data with latitude/longitude coordinates
  • Columns: Lat_Public, Long_Public (required for spatial analysis)
  • Optional: Date/Time, Crime Type, District fields

───────────────────────────────────────────────────────────────────────────────
2. DATA LOADING
───────────────────────────────────────────────────────────────────────────────

THREE WAYS TO LOAD DATA:

a) Upload File:
   • Click "Upload File" button
   • Supported formats: CSV, XLSX, GeoJSON
   • File must contain Lat_Public and Long_Public columns

b) Load Default Dataset:
   • Click "Load Default Dataset" button
   • Loads sample CrimeIncidents.xlsx file
   • Use this for testing and learning

c) Load from URL:
   • Enter CSV URL in the text box
   • Click "Load from URL" button
   • URL must be publicly accessible

───────────────────────────────────────────────────────────────────────────────
3. DATA CLEANING
───────────────────────────────────────────────────────────────────────────────

SELECT CLEANING METHOD from dropdown:

- None: Use data as-is
- Drop NA: Remove rows with missing values
- Fill NA with 0: Replace missing values with zero
- Remove Duplicates: Delete duplicate rows

Cleaning is applied automatically when you run Explore or Analysis.

───────────────────────────────────────────────────────────────────────────────
4. DATA EXPLORATION
───────────────────────────────────────────────────────────────────────────────

Click "Explore Data" button to see:

- Dataset shape (rows × columns)
- Column data types
- Statistical summary
- Missing value counts
- Duplicate row counts
- Coordinate validity percentage

───────────────────────────────────────────────────────────────────────────────
5. VISUALIZATION
───────────────────────────────────────────────────────────────────────────────

Click "Generate Visualizations" to create:

- Distribution plots for numeric variables
- Bar charts for categorical variables
- Correlation heatmaps
- Time series plots (if date column exists)

───────────────────────────────────────────────────────────────────────────────
6. ANALYSIS METHODS
───────────────────────────────────────────────────────────────────────────────

SELECT ANALYSIS METHOD from dropdown:

a) Grid-based Hotspot (250m):
   • Fine-grained spatial analysis
   • BEST FOR: Neighborhood-level patterns

b) Grid-based Hotspot (500m):
   • Broader spatial analysis
   • BEST FOR: District-level analysis

c) Temporal Analysis:
   • Analyzes patterns by hour, day, month
   • REQUIRES: Date/Time column

d) Crime Type Distribution:
   • Shows frequency of each crime type
   • REQUIRES: Crime type column

e) Kernel Density Estimation:
   • Advanced statistical method (placeholder)

───────────────────────────────────────────────────────────────────────────────
7. REPORT GENERATION
───────────────────────────────────────────────────────────────────────────────

GENERATE REPORT:
  • Click "Generate Report" button
  • Comprehensive text report appears on screen

SAVE REPORT TO FILE:
  • Click "Save Report to File" button
  • File saved as: crime_analysis_report_YYYYMMDD_HHMMSS.txt

───────────────────────────────────────────────────────────────────────────────
8. SECURITY FEATURES
───────────────────────────────────────────────────────────────────────────────

BUILT-IN SECURITY:
- Local Processing: All data stays on your machine
- Security Event Logging: All actions are logged
- Data Validation: Checks for sensitive columns

SECURITY OPTIONS:
☐ Encrypt saved files
☐ Anonymize coordinates

───────────────────────────────────────────────────────────────────────────────
9. TESTING
───────────────────────────────────────────────────────────────────────────────

Click "Run All Tests" to verify system functionality.

TESTS PERFORMED:
1. Data Loading Functions
2. Data Validation
3. Data Cleaning Operations
4. Coordinate Processing
5. Hotspot Detection Algorithm
6. Visualization Generation
7. Report Generation
8. Security Features

───────────────────────────────────────────────────────────────────────────────
10. TROUBLESHOOTING
───────────────────────────────────────────────────────────────────────────────

COMMON ISSUES:

❌ "No data loaded"
   SOLUTION: Click "Load Default Dataset" or upload a file first

❌ "Missing Lat_Public/Long_Public columns"
   SOLUTION: Ensure your data has these exact column names

❌ "No valid coordinates found"
   SOLUTION: Use "Drop NA" cleaning option

❌ "No module named geopandas"
   SOLUTION: Install geopandas: conda install -c conda-forge geopandas

❌ Plots not appearing
   SOLUTION: Scroll down in output panel or restart kernel

❌ Report save failed
   SOLUTION: Check file permissions and disk space

═══════════════════════════════════════════════════════════════════════════════
                              END OF USER GUIDE
═══════════════════════════════════════════════════════════════════════════════
        """
        print(help_content)

print("✅ Help system defined!")

✅ Help system defined!


In [211]:
# CONNECT WIDGETS TO FUNCTIONS AND CREATE UI

# Connect all widgets to their callback functions
upload.observe(handle_upload, names='value')
load_default_button.on_click(load_default)
load_url_button.on_click(load_url)
explore_button.on_click(explore)
visualize_button.on_click(visualize)
hotspot_button.on_click(hotspot_analysis)
generate_report_button.on_click(generate_report)
save_report_button.on_click(save_report)
run_tests_button.on_click(run_all_tests)
help_button.on_click(show_help)

# Create tabbed interface
tab_contents = [
    # Tab 1: Data Management
    VBox([
        HTML("<h3>📁 Data Management</h3>"),
        HBox([upload, load_default_button]),
        output_load,
        HTML("<h4>Or load from URL:</h4>"),
        HBox([url_box, load_url_button]),
        HTML("<h4>Data Cleaning:</h4>"),
        HBox([cleaning_dropdown]),
    ]),
    
    # Tab 2: Exploration & Analysis
    VBox([
        HTML("<h3>🔍 Data Exploration</h3>"),
        HBox([explore_button, visualize_button]),
        output_explore,
        output_visualize,
    ]),
    
    # Tab 3: Hotspot Analysis
    VBox([
        HTML("<h3>📍 Hotspot Analysis</h3>"),
        HTML("<p>Select analysis method and click 'Run Analysis'</p>"),
        HBox([analysis_dropdown, hotspot_button]),
        output_hotspot,
    ]),
    
    # Tab 4: Reports
    VBox([
        HTML("<h3>📊 Reports</h3>"),
        HBox([generate_report_button, save_report_button]),
        output_reports,
    ]),
    
    # Tab 5: Testing
    VBox([
        HTML("<h3>🧪 System Testing</h3>"),
        HTML("<p>Run comprehensive tests to verify all functions work correctly</p>"),
        run_tests_button,
        output_testing,
    ]),
    
    # Tab 6: Security
    VBox([
        HTML("<h3>🔒 Security & Privacy</h3>"),
        HTML("<p>Security options and event log</p>"),
        encrypt_checkbox,
        anonymize_checkbox,
        output_security,
    ]),
    
    # Tab 7: Help
    VBox([
        HTML("<h3>❓ Help & Documentation</h3>"),
        help_button,
        output_help,
    ]),
]

tab = Tab(children=tab_contents)
tab.set_title(0, '📁 Data')
tab.set_title(1, '🔍 Explore')
tab.set_title(2, '📍 Analysis')
tab.set_title(3, '📊 Reports')
tab.set_title(4, '🧪 Testing')
tab.set_title(5, '🔒 Security')
tab.set_title(6, '❓ Help')

print("✅ UI layout created!")

✅ UI layout created!


In [212]:
# DISPLAY THE APPLICATION

# Display title and application
display(HTML("<h1 style='text-align: center; color: #2C3E50;'>🚔 Crime Hotspot Analysis Tool v2.0</h1>"))
display(HTML("<p style='text-align: center; color: #7F8C8D;'>Comprehensive Data Product for Law Enforcement Analysis</p>"))
display(HTML("<p style='text-align: center; color: #95A5A6; font-size: 0.9em;'>Author: Chynara Gambarova | CST-580 | December 2025</p>"))
display(tab)

print("\n" + "="*80)
print("APPLICATION READY!")
print("="*80)
print("\n🎯 Quick Start Guide:")
print("  1. Go to '📁 Data' tab and click 'Load Default Dataset'")
print("  2. Go to '🔍 Explore' tab and click 'Explore Data'")
print("  3. Go to '📍 Analysis' tab, select method, and click 'Run Analysis'")
print("  4. Go to '📊 Reports' tab and click 'Generate Report'")
print("  5. Go to '🧪 Testing' tab and click 'Run All Tests'")
print("\n💡 Need help? Click the '❓ Help' tab!")
print("="*80)

HTML(value="<h1 style='text-align: center; color: #2C3E50;'>🚔 Crime Hotspot Analysis Tool v2.0</h1>")

HTML(value="<p style='text-align: center; color: #7F8C8D;'>Comprehensive Data Product for Law Enforcement Anal…

HTML(value="<p style='text-align: center; color: #95A5A6; font-size: 0.9em;'>Author: Chynara Gambarova | CST-5…


APPLICATION READY!

🎯 Quick Start Guide:
  1. Go to '📁 Data' tab and click 'Load Default Dataset'
  2. Go to '🔍 Explore' tab and click 'Explore Data'
  3. Go to '📍 Analysis' tab, select method, and click 'Run Analysis'
  4. Go to '📊 Reports' tab and click 'Generate Report'
  5. Go to '🧪 Testing' tab and click 'Run All Tests'

💡 Need help? Click the '❓ Help' tab!
